# Collecting Data From NBA API

In [61]:
# Import Libraries
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [3]:
# Get Player IDs of all Players

all_players = players.get_active_players()

print(len(all_players))

571


In [ ]:
# Get the career avg data of all players

df = pd.DataFrame()
iteration = 1

for player in all_players:
    try:
        print(f'Iteration: {iteration}')
        player_id = player.get('id', None)

        time.sleep(2)
    
        if player_id is not None:
            player_object = playercareerstats.PlayerCareerStats(per_mode36= 'PerGame', player_id= str(player_id))
            df_player = player_object.get_data_frames()[0]
            df = pd.concat([df, df_player])
    
    except KeyError as e: print(f'Skipping Player ID: {player_id}')
    except Exception as e: print(f'Trouble getting player {player_id}')

    iteration += 1
 
df
df.to_csv('../Datasets/nba.csv')

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15


/var/folders/kp/0yh04nk14gq26qb__8kstckh0000gn/T/ipykernel_19190/1814564361.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_player])


Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
Iteration: 73
Iteration: 74
Iteration: 75
Iteration: 76
Iteration: 77
Iteration: 78
Iteration: 79
Iteration: 80
Iteration: 81
Iteration: 82
Iteration: 83
Iteration: 84
Iteration: 85
Iteration: 86
Iterat

In [4]:
# Get the career total data of all players

df_totals = pd.DataFrame()
failed_totals = []
iteration = 1

for player in all_players:
    try:
        print(f'Iteration: {iteration}')
        player_id = player.get('id', None)

        time.sleep(2)
    
        if player_id is not None:
            player_object = playercareerstats.PlayerCareerStats(per_mode36= 'Totals', player_id= str(player_id))
            df_player = player_object.get_data_frames()[0]
            df_totals = pd.concat([df_totals, df_player])
    
    except KeyError as e: 
        print(f'Skipping Player ID: {player_id}')
        failed_totals.append(player_id)
    except Exception as e: 
        print(f'Trouble getting player {player_id}')
        failed_totals.append(player_id)

    iteration += 1
 
df_totals
df_totals.to_csv('../Datasets/nba_totals.csv')

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15


/var/folders/kp/0yh04nk14gq26qb__8kstckh0000gn/T/ipykernel_35371/2882352342.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_totals = pd.concat([df_totals, df_player])


Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
Iteration: 73
Iteration: 74
Iteration: 75
Iteration: 76
Iteration: 77
Iteration: 78
Iteration: 79
Iteration: 80
Iteration: 81
Iteration: 82
Iteration: 83
Iteration: 84
Iteration: 85
Iteration: 86
Iterat

In [62]:
df = pd.read_csv('../Datasets/nba.csv')
df = df.drop(['Unnamed: 0'], axis= 1)
df

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1630173,2020-21,0,1610612748,MIA,21.0,61,4,12.1,2.0,...,0.509,1.2,2.2,3.4,0.5,0.3,0.5,0.7,1.5,5.0
1,1630173,2021-22,0,1610612761,TOR,22.0,73,28,23.6,3.6,...,0.595,2.0,4.5,6.5,1.1,0.5,0.6,1.2,2.1,9.1
2,1630173,2022-23,0,1610612761,TOR,23.0,55,12,20.7,3.6,...,0.702,1.8,4.1,6.0,0.9,0.6,0.5,1.1,1.9,9.2
3,1630173,2023-24,0,1610612761,TOR,24.0,25,0,17.5,3.1,...,0.571,2.0,3.4,5.4,1.8,0.6,0.5,1.2,1.6,7.7
4,1630173,2023-24,0,1610612752,NYK,24.0,49,18,24.2,3.2,...,0.643,2.9,4.3,7.2,1.1,0.6,1.1,1.1,2.1,7.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3945,203967,2022-23,0,1610612760,OKC,29.0,20,0,13.7,2.6,...,0.844,0.7,2.6,3.3,0.9,0.4,0.1,1.0,1.5,7.4
3946,203967,2022-23,0,0,TOT,29.0,57,12,14.1,2.2,...,0.829,0.9,2.7,3.6,1.3,0.4,0.1,1.0,1.7,6.4
3947,203967,2023-24,0,1610612744,GSW,30.0,64,9,17.2,2.8,...,0.849,1.1,3.3,4.4,2.3,0.5,0.2,1.2,1.8,8.0
3948,203967,2024-25,0,1610612743,DEN,31.0,16,4,13.1,1.3,...,0.700,0.9,2.3,3.1,1.4,0.4,0.1,0.9,1.2,3.5


In [86]:
df_totals = pd.read_csv('../Datasets/nba_totals.csv')
df_totals = df_totals.drop('Unnamed: 0', axis = 1)
df_totals = df_totals.loc[:, ['PLAYER_ID', 'GP', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PTS']]
df_totals = df_totals.groupby(['PLAYER_ID']).sum()
df_totals['FG_PCT'] = np.round(df_totals['FGM'] / df_totals['FGA'], 3)
df_totals['3_PCT'] = np.round(df_totals['FG3M'] / df_totals['FG3A'], 3)
df_totals['FT_PCT'] = np.round(df_totals['FTM'] / df_totals['FTA'], 3)
df_totals = df_totals.drop(['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA'], axis = 1)

for i in df_totals.loc[:, 'REB':'PTS'].columns:
    df_totals[i] = np.round(df_totals[i] / df_totals['GP'], 1)

df_totals.head()



,GP,REB,AST,STL,BLK,TOV,PTS,FG_PCT,3_PCT,FT_PCT
PLAYER_ID,,,,,,,,,,
2544,1562,7.5,7.4,1.5,0.7,3.5,27.0,0.506,0.349,0.737
101108,1359,4.4,9.2,2.0,0.2,2.3,16.9,0.469,0.370,0.871
200768,1311,4.1,5.8,1.2,0.3,2.2,13.3,0.424,0.367,0.815
200782,1050,5.3,1.3,1.1,0.3,0.9,6.3,0.422,0.364,0.753
201142,1175,7.0,4.4,1.0,1.1,3.2,27.3,0.504,0.390,0.883


In [88]:
failed_names = []
player_names = np.array([])

for i in range(len(df_totals.index)-1):
    try:
        print(f'Iteration: {i+1}')
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=str(df_totals.index[i]))
        player_info = player_info.get_data_frames()[0] 
        player_names = np.append(player_names, player_info['DISPLAY_FIRST_LAST'][0])
        time.sleep(2)
    except KeyError as e: 
        print(f'Skipping Player ID: {df_totals.index[i]}')
        failed_totals.append(df_totals.index[i])
    except Exception as e: 
        print(f'Trouble getting player {df_totals.index[i]}')
        failed_totals.append(df_totals.index[i])    

df_totals['NAMES'] = player_names

df_totals.to_csv('../Datasets/nba_career_avgs.csv')

print('done')

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
I

KeyboardInterrupt: 

In [82]:
from nba_api.stats.endpoints import commonplayerinfo

player_info = commonplayerinfo.CommonPlayerInfo(player_id='2544')

player_info = player_info.get_data_frames()[0]['DISPLAY_FIRST_LAST'][0]

player_info = np.array([])

player_info = np.append(player_info, [1, 2])

print(player_info)

print(range(5))

[1. 2.]
range(0, 5)
